In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
import math
import random
import numpy as np
import pandas as pd
import scipy.io as sio
from sklearn.model_selection import train_test_split
from tensorflow.python.keras.utils.np_utils import to_categorical
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from imblearn.metrics import geometric_mean_score
from sklearn.metrics import confusion_matrix,accuracy_score,recall_score,precision_score,f1_score,roc_auc_score
import warnings
warnings.filterwarnings("ignore")

In [2]:
data1 = pd.read_csv("../../dataset/NASA/MC2.csv")
data1.head()

,LOC_BLANK numeric,BRANCH_COUNT numeric,CALL_PAIRS numeric,LOC_CODE_AND_COMMENT numeric,LOC_COMMENTS numeric,CONDITION_COUNT numeric,CYCLOMATIC_COMPLEXITY numeric,CYCLOMATIC_DENSITY numeric,DECISION_COUNT numeric,DECISION_DENSITY numeric,...,NODE_COUNT numeric,NORMALIZED_CYLOMATIC_COMPLEXITY numeric,NUM_OPERANDS numeric,NUM_OPERATORS numeric,NUM_UNIQUE_OPERANDS numeric,NUM_UNIQUE_OPERATORS numeric,NUMBER_OF_LINES numeric,PERCENT_COMMENTS numeric,LOC_TOTAL numeric,Defective
0,2,3,0,2,7,4,2,0.18,2,2.0,...,6,0.09,11,15,6,8,22,50.00,11,N
1,1,3,1,1,0,4,2,0.13,2,2.0,...,7,0.11,12,19,8,10,18,6.67,15,N
2,6,5,1,4,5,8,3,0.20,4,2.0,...,10,0.10,25,38,12,11,29,45.00,15,N
3,6,5,1,4,3,8,3,0.20,4,2.0,...,10,0.12,26,38,10,11,26,38.89,15,N
4,2,3,0,0,1,4,2,0.29,2,2.0,...,6,0.17,7,13,5,9,12,12.50,7,N


In [3]:
data1.describe()

,LOC_BLANK numeric,BRANCH_COUNT numeric,CALL_PAIRS numeric,LOC_CODE_AND_COMMENT numeric,LOC_COMMENTS numeric,CONDITION_COUNT numeric,CYCLOMATIC_COMPLEXITY numeric,CYCLOMATIC_DENSITY numeric,DECISION_COUNT numeric,DECISION_DENSITY numeric,...,MULTIPLE_CONDITION_COUNT numeric,NODE_COUNT numeric,NORMALIZED_CYLOMATIC_COMPLEXITY numeric,NUM_OPERANDS numeric,NUM_OPERATORS numeric,NUM_UNIQUE_OPERANDS numeric,NUM_UNIQUE_OPERATORS numeric,NUMBER_OF_LINES numeric,PERCENT_COMMENTS numeric,LOC_TOTAL numeric
count,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,...,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000,127.000000
mean,11.070866,16.299213,2.212598,2.346457,13.559055,27.118110,8.834646,0.269606,12.346457,2.105827,...,13.582677,27.960630,0.169606,85.598425,116.574803,23.622047,15.472441,70.425197,21.014803,43.330709
std,16.195816,21.586414,3.233333,4.488748,21.199601,37.197588,10.947757,0.179706,16.077381,0.203456,...,18.667704,32.964965,0.114901,104.109547,146.754277,20.971231,6.642608,86.123345,15.507606,52.489009
min,0.000000,3.000000,0.000000,0.000000,0.000000,4.000000,2.000000,0.040000,2.000000,2.000000,...,2.000000,5.000000,0.030000,1.000000,3.000000,1.000000,3.000000,4.000000,0.000000,2.000000
25%,1.000000,5.000000,0.000000,0.000000,0.000000,7.000000,3.000000,0.170000,3.000000,2.000000,...,3.500000,9.000000,0.100000,18.500000,24.000000,9.000000,11.000000,17.000000,8.110000,11.500000
50%,6.000000,9.000000,1.000000,0.000000,6.000000,16.000000,5.000000,0.220000,6.000000,2.000000,...,8.000000,18.000000,0.140000,50.000000,65.000000,17.000000,15.000000,41.000000,22.220000,26.000000
75%,13.000000,19.000000,3.000000,3.000000,16.500000,29.000000,10.000000,0.295000,14.000000,2.155000,...,14.500000,31.000000,0.195000,120.000000,153.500000,33.000000,20.000000,83.500000,30.495000,53.500000
max,105.000000,125.000000,18.000000,19.000000,123.000000,216.000000,63.000000,1.000000,102.000000,3.000000,...,108.000000,191.000000,0.670000,615.000000,864.000000,98.000000,32.000000,474.000000,70.000000,306.000000


In [4]:
data1.values.shape

(127, 40)

In [5]:
data2=data1.values
for i in range(127):
    if data2[i][39]=="Y":
        data2[i][39]=1
    else:
        data2[i][39]=0
data2[1]

array([1, 3, 1, 1, 0, 4, 2, 0.13, 2, 2.0, 1, 0.5, 7, 1, 0.0, 14, 0, 2,
       1.0, 17.24, 7.5, 969.51, 0.04, 31, 0.13, 53.86, 129.27, 0.5, 1, 2,
       7, 0.11, 12, 19, 8, 10, 18, 6.67, 15, 0], dtype=object)

In [6]:
X=np.array(data2).astype(float)  

In [7]:
X[:,:39]

array([[  2.  ,   3.  ,   0.  , ...,  22.  ,  50.  ,  11.  ],
       [  1.  ,   3.  ,   1.  , ...,  18.  ,   6.67,  15.  ],
       [  6.  ,   5.  ,   1.  , ...,  29.  ,  45.  ,  15.  ],
       ...,
       [ 19.  ,  13.  ,   3.  , ..., 108.  ,  34.48,  68.  ],
       [  5.  ,   3.  ,   3.  , ...,  35.  ,  35.71,  25.  ],
       [ 23.  ,  15.  ,   3.  , ..., 137.  ,  33.04,  85.  ]])

In [8]:
y=X[:,39]

In [9]:
X=X[:,0:39]

In [10]:
from imblearn.over_sampling import RandomOverSampler,SMOTE
#os =  RandomOverSampler()
X_res, y_res = SMOTE().fit_resample(X, y)
X_res.shape,y_res.shape

((166, 39), (166,))

In [11]:
X_res.shape
#X_res=X_res.reshape(898,21,1)
#X_res.shape

(166, 39)

In [12]:
import numpy as np
from sklearn import preprocessing
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import datetime
import csv
import time

def show_accuracy(predictLabel,Label):
    Label = np.ravel(Label).tolist()
    predictLabel = predictLabel.tolist()
    count = 0
    for i in range(len(Label)):
        if Label[i] == predictLabel[i]:
            count += 1
    return (round(count/len(Label),5))

class node_generator(object):
    def __init__(self, whiten = False):
        self.Wlist = []
        self.blist = []
        self.function_num = 0
        self.whiten = whiten

    def sigmoid(self, x):
        return 1.0/(1 + np.exp(-x))

    def relu(self, x):
        return np.maximum(x, 0)

    def tanh(self, x):
        return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))

    def linear(self, x):
        return x

    def orth(self, W):
        """
        目前看来，这个函数应该配合下一个generator函数是生成权重的
        """
        for i in range(0, W.shape[1]):
            w = np.mat(W[:,i].copy()).T
            w_sum = 0
            for j in range(i):
                wj = np.mat(W[:,j].copy()).T
                w_sum += (w.T.dot(wj))[0,0]*wj
            w -= w_sum
            w = w/np.sqrt(w.T.dot(w))
            W[:,i] = np.ravel(w)

        return W

    def generator(self, shape, times):
        for i in range(times):
            random.seed(i)
            W = 2*np.random.random(size=shape)-1
            if self.whiten == True:
                W = self.orth(W)   # 只在增强层使用
            b = 2*np.random.random() -1
            yield (W, b)

    def generator_nodes(self, data, times, batchsize, function_num):
        # 按照bls的理论，mapping layer是输入乘以不同的权重加上不同的偏差之后得到的
        # 若干组，所以，权重是一个列表，每一个元素可作为权重与输入相乘
        self.Wlist = [elem[0] for elem in self.generator((data.shape[1], batchsize), times)]
        self.blist = [elem[1] for elem in self.generator((data.shape[1], batchsize), times)]

        self.function_num = {'linear':self.linear,
                        'sigmoid': self.sigmoid,
                        'tanh':self.tanh,
                        'relu':self.relu }[function_num]  # 激活函数供不同的层选择
        # 下面就是先得到一组mapping nodes，再不断叠加，得到len(Wlist)组mapping nodes
        nodes = self.function_num(data.dot(self.Wlist[0]) + self.blist[0])
        for i in range(1, len(self.Wlist)):
            nodes = np.column_stack((nodes, self.function_num(data.dot(self.Wlist[i])+self.blist[i])))
        return nodes

    def transform(self,testdata):
        testnodes = self.function_num(testdata.dot(self.Wlist[0])+self.blist[0])
        for i in range(1,len(self.Wlist)):
            testnodes = np.column_stack((testnodes, self.function_num(testdata.dot(self.Wlist[i])+self.blist[i])))
        return testnodes

class scaler:
    def __init__(self):
        self._mean = 0
        self._std = 0
    
    def fit_transform(self,traindata):
        self._mean = traindata.mean(axis = 0)
        self._std = traindata.std(axis = 0)
        return (traindata-self._mean)/(self._std+0.001)
    
    def transform(self,testdata):
        return (testdata-self._mean)/(self._std+0.001)

class broadnet:
    def __init__(self, 
                 maptimes = 10, 
                 enhencetimes = 10,
                 map_function = 'linear',
                 enhence_function = 'linear',
                 batchsize = 'auto', 
                 reg = 0.001):
        
        self._maptimes = maptimes
        self._enhencetimes = enhencetimes
        self._batchsize = batchsize
        self._reg = reg
        self._map_function = map_function
        self._enhence_function = enhence_function
        
        self.W = 0
        self.pesuedoinverse = 0
        self.normalscaler = scaler()
        self.onehotencoder = preprocessing.OneHotEncoder(sparse = False)
        self.mapping_generator = node_generator()
        self.enhence_generator = node_generator(whiten = True)

    def fit(self,data,label,weight):
        if self._batchsize == 'auto':
            self._batchsize = data.shape[1]
        data = self.normalscaler.fit_transform(data)
        label = self.onehotencoder.fit_transform(np.mat(label).T)
        
        mappingdata = self.mapping_generator.generator_nodes(data,self._maptimes,self._batchsize,self._map_function)
        enhencedata = self.enhence_generator.generator_nodes(mappingdata,self._enhencetimes,self._batchsize,self._enhence_function)
        
        #print('number of mapping nodes {0}, number of enhence nodes {1}'.format(mappingdata.shape[1],enhencedata.shape[1]))
        #print('mapping nodes maxvalue {0} minvalue {1} '.format(round(np.max(mappingdata),5),round(np.min(mappingdata),5)))
        #print('enhence nodes maxvalue {0} minvalue {1} '.format(round(np.max(enhencedata),5),round(np.min(enhencedata),5)))
        
        inputdata = np.column_stack((mappingdata,enhencedata))
        pesuedoinverse = self.pinv(inputdata,self._reg,weight)
        self.W =  pesuedoinverse.dot(label)
        
        #print('W:', self.W)
        #print('W:', self.W.shape)  
    
    #改写伪逆矩阵算法，将权重输入
    def pinv(self,A,reg,weight):
        return np.mat(reg*np.eye(A.shape[1])+A.T.dot(weight).dot(A)).I.dot(A.T).dot(weight)
    
    def decode(self,Y_onehot):
        Y = []
        for i in range(Y_onehot.shape[0]):
            lis = np.ravel(Y_onehot[i,:]).tolist()
            Y.append(lis.index(max(lis)))
        return np.array(Y)
    
    def accuracy(self,predictlabel,label):
        label = np.ravel(label).tolist()
        predictlabel = predictlabel.tolist()
        count = 0
        for i in range(len(label)):
            if label[i] == predictlabel[i]:
                count += 1
        return (round(count/len(label),5))
        
    def predict(self,testdata):
        testdata = self.normalscaler.transform(testdata)
        test_mappingdata = self.mapping_generator.transform(testdata)
        test_enhencedata = self.enhence_generator.transform(test_mappingdata)
        
        test_inputdata = np.column_stack((test_mappingdata,test_enhencedata)) 
        #print('*predictlabel shape:',self.decode(test_inputdata.dot(self.W)).shape)
        #print('*predictlabel:', self.decode(test_inputdata.dot(self.W)))
        #print('*accuracy:',show_accuracy(self.decode(test_inputdata.dot(self.W)),testlabel))
        return self.decode(test_inputdata.dot(self.W))
    
    def addWeight(self,trainlabel):
        #对BLS设置加权，按照训练集中的label比例进行权重设置
        #求训练数据集中label为1的个数
        count=0
        for z in range(len(trainlabel)):
            #print ('个数 %d ' %z)
            #print('k_train_label:', k_train_label[z])
            if trainlabel[z]==1:
                count=count+1
        #print ('label为1的个数 %d ' %j)
        #print(count)
        #print(k_train_label.shape)
        #print(len(k_train_label))
        #权重设置 
        weight = np.zeros((len(trainlabel),len(trainlabel)))
        for i in range(len(trainlabel)):
            if trainlabel[i]==1:
                #weight[i,i]=0.618/count
                weight[i,i]=2*(len(trainlabel)-count)/len(trainlabel)
            else:
                #weight[i,i]=1/(len(k_train_label)-count)
                weight[i,i]=2*count/len(trainlabel)
        #print('Weight:', weight)
        return weight

In [13]:
#将数据集划分为训练集和测试集
#traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.2,random_state = 0)
#print(traindata.shape,trainlabel.shape,testdata.shape,testlabel.shape)
#X_train, X_test, y_train, y_test = train_test_split(X_res, y_res, test_size=0.2)
#y_train = np_utils.to_categorical(y_train)
#y_test = np_utils.to_categorical(y_test)

In [14]:
Evaluat_list=[]
for i in np.arange(0,10):
    #将数据集划分为训练集和测试集
    traindata,testdata,trainlabel,testlabel = train_test_split(X_res,y_res,test_size=0.25,random_state = i)
    bls = broadnet(maptimes = 20, 
               enhencetimes = 15,
               map_function = 'relu',
               enhence_function = 'relu',
               batchsize =100,
               reg = 0.001)

    #根据训练数据设置权重
    weight=bls.addWeight(trainlabel)

    #训练
    starttime = datetime.datetime.now()
    bls.fit(traindata,trainlabel,weight)
    endtime = datetime.datetime.now()
    trainingTime=(endtime - starttime).total_seconds()
    #print('the training time of BLS is {0} seconds'.format((endtime - starttime).total_seconds()))

    #预测
    predictlabel = bls.predict(testdata)

    #评价指标计算
    acc=accuracy_score(testlabel,predictlabel,normalize=True)
    fmeasure=f1_score(testlabel,predictlabel, average='weighted', labels=np.unique(testlabel))
    try:
        auc=roc_auc_score(testlabel,predictlabel, average='macro', sample_weight=None)
    except ValueError:
        pass
    MCC=matthews_corrcoef(testlabel,predictlabel)
    Gmeasure=geometric_mean_score(testlabel,predictlabel, average='weighted')
    recall=recall_score(testlabel, predictlabel, average='weighted')

    print('trainingTime：%f,acc：%f,fmeasure：%f,auc：%f,recall：%f,MCC：%f,Gmeasure：%f'%(trainingTime,acc,fmeasure,auc,recall,MCC,Gmeasure))
    Evaluat_tuple=(trainingTime,acc,fmeasure,auc,recall,MCC,Gmeasure)    
    Evaluat_list.append(Evaluat_tuple)

# 表头
header = ['trainTime', 'acc', 'fmeasure', 'auc','recall', 'MCC', 'Gmeasure']
with open('./data_remember/0.75WBLS_Smote_MC2_Evaluat.csv', 'w', encoding='utf-8', newline='') as file_obj:
    # 创建对象
    writer = csv.writer(file_obj)
    # 写表头
    writer.writerow(header)
    # 3.写入数据(一次性写入多行)
    writer.writerows(Evaluat_list)


    #sio.savemat('./data_remember/WBLS_Smote_CM1_test.mat',{'time':format((endtime - starttime).total_seconds()),'acc':acc,'f1':fmeasure,'auc':auc,'mcc':MCC,'Gm':Gmeasure})

trainingTime：2.452242,acc：0.928571,fmeasure：0.927697,auc：0.921053,recall：0.928571,MCC：0.863099,Gmeasure：0.921022
trainingTime：2.464137,acc：0.833333,fmeasure：0.833239,auc：0.833333,recall：0.833333,MCC：0.667424,Gmeasure：0.833333
trainingTime：2.456112,acc：0.952381,fmeasure：0.952381,auc：0.952381,recall：0.952381,MCC：0.904762,Gmeasure：0.952381
trainingTime：2.454330,acc：0.904762,fmeasure：0.904762,auc：0.903890,recall：0.904762,MCC：0.807780,Gmeasure：0.903890
trainingTime：2.467993,acc：0.785714,fmeasure：0.779985,auc：0.772311,recall：0.785714,MCC：0.575034,Gmeasure：0.772195
trainingTime：2.449159,acc：0.928571,fmeasure：0.927426,auc：0.916667,recall：0.928571,MCC：0.860663,Gmeasure：0.916589
trainingTime：2.464285,acc：0.880952,fmeasure：0.880610,auc：0.877574,recall：0.880952,MCC：0.759506,Gmeasure：0.877568
trainingTime：2.451705,acc：0.904762,fmeasure：0.904762,auc：0.906818,recall：0.904762,MCC：0.813636,Gmeasure：0.906816
trainingTime：2.457641,acc：0.833333,fmeasure：0.833618,auc：0.834096,recall：0.833333,MCC：0.665910,G